In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.hopsworks_connections import pull_last_row, upload_data, pull_model, pull_data

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Get model from our Hopsworks model registry

In [10]:
MODEL_NAME = 'random_forest_ohlc' # this one predicts tmw_1_0_percent_increase_binary

# ***** later implement code that checks the latest model version and pulls that one
# def pull_model(model_name, model_version):
model = pull_model(MODEL_NAME, 1)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


### Get the latest day of ethereum data

In [52]:
eth_yesterday = pull_data('eth_ohlc_transformed', 2, 'eth_ohlc_transformed_view', 2)[-1:]

eth_yesterday

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (1.81s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
3061,2024-04-22 00:00:00+00:00,3146.652021,3232.067831,3134.798198,3198.484468,1.183567e+10,3.706587e+06,3.834176e+11,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0


### Define predictors, the input data points, and get a prediction for tmw_1_0_percent_increase_binary

In [53]:
predictors = []
periods = [2, 5, 10, 25, 50, 100]
percent_increase_counts = ['1_0', '1_25', '1_5', '1_75', '2_0', '2_25', '2_5', '2_75', '3_0']
for period in periods:
    for indicator in ['ema', 'rsi', 'sma']:
        predictors.append(f'{indicator}_{period}')
    for percent in percent_increase_counts:
        predictors.append(f'last_{period}_day_{percent}_percent_increase_count')
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
predictors.extend(months)
weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
predictors.extend(weekdays)

x0 = eth_yesterday[predictors]
x0

,ema_2,rsi_2,sma_2,last_2_day_1_0_percent_increase_count,last_2_day_1_25_percent_increase_count,last_2_day_1_5_percent_increase_count,last_2_day_1_75_percent_increase_count,last_2_day_2_0_percent_increase_count,last_2_day_2_25_percent_increase_count,last_2_day_2_5_percent_increase_count,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
3061,1.006062,90.123553,1.008092,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


In [54]:
prediction = model.predict(x0)
prediction

array([0.])

In [55]:
pred = int(prediction[0])
eth_yesterday['pred_tmw_1_0_percent_increase_binary'] = pred

eth_yesterday['pred_tmw_1_0_percent_increase_binary'] = eth_yesterday['pred_tmw_1_0_percent_increase_binary'].astype('Int64')

### Upload this prediction to a separate predictions feature group in Hopsworks

In [45]:
print(eth_yesterday.columns.tolist())

['date', 'open', 'high', 'low', 'close', 'volume', 'volume_eth', 'market_cap', 'tmw_avg_high_close', 'tmw_percent_increase', 'tmw_1_0_percent_increase_binary', 'tmw_1_25_percent_increase_binary', 'tmw_1_5_percent_increase_binary', 'tmw_1_75_percent_increase_binary', 'tmw_2_0_percent_increase_binary', 'tmw_2_25_percent_increase_binary', 'tmw_2_5_percent_increase_binary', 'tmw_2_75_percent_increase_binary', 'tmw_3_0_percent_increase_binary', 'tmw_positive_percent_increase_binary', 'ema_2', 'rsi_2', 'sma_2', 'last_2_day_1_0_percent_increase_count', 'last_2_day_1_25_percent_increase_count', 'last_2_day_1_5_percent_increase_count', 'last_2_day_1_75_percent_increase_count', 'last_2_day_2_0_percent_increase_count', 'last_2_day_2_25_percent_increase_count', 'last_2_day_2_5_percent_increase_count', 'last_2_day_2_75_percent_increase_count', 'last_2_day_3_0_percent_increase_count', 'ema_5', 'rsi_5', 'sma_5', 'last_5_day_1_0_percent_increase_count', 'last_5_day_1_25_percent_increase_count', 'last_

In [57]:
vars = ['date', 'open', 'high', 'low', 'close', 'volume', 'volume_eth', 'market_cap', 'pred_tmw_1_0_percent_increase_binary']
eth_yesterday = eth_yesterday[vars]

eth_yesterday

,date,open,high,low,close,volume,volume_eth,market_cap,pred_tmw_1_0_percent_increase_binary
3061,2024-04-22 00:00:00+00:00,3146.652021,3232.067831,3134.798198,3198.484468,1.183567e+10,3.706587e+06,3.834176e+11,0


In [62]:
eth_yesterday.dtypes

date                                    datetime64[ns, UTC]
open                                                float64
high                                                float64
low                                                 float64
close                                               float64
volume                                              float64
volume_eth                                          float64
market_cap                                          float64
pred_tmw_1_0_percent_increase_binary                  Int64
dtype: object

In [68]:
from datetime import timedelta

def add_fake_observations(df, x):
    if len(df) != 1:
        raise ValueError("Input dataframe must have exactly one row.")
    
    # Extract the date from the input dataframe and convert to datetime if necessary
    base_date = pd.to_datetime(df.iloc[0]['date'])
    
    # Create a new DataFrame with x rows, decrementing the date for each new row
    new_dates = [base_date - timedelta(days=i) for i in range(1, x+1)]
    new_data = pd.DataFrame(new_dates, columns=['date'])
    
    # Replicate the original row x times and reset the index
    replicated_data = pd.concat([df]*x, ignore_index=True)
    
    # Replace the dates in the replicated data with the new dates
    replicated_data['date'] = new_data['date']
    
    # Concatenate the original data with the new fake observations
    result = pd.concat([df, replicated_data], ignore_index=True)
    
    return result

bigger_preds = add_fake_observations(eth_yesterday, 3)
bigger_preds

,date,open,high,low,close,volume,volume_eth,market_cap,pred_tmw_1_0_percent_increase_binary
0,2024-04-22 00:00:00+00:00,3146.652021,3232.067831,3134.798198,3198.484468,1.183567e+10,3.706587e+06,3.834176e+11,0
1,2024-04-21 00:00:00+00:00,3146.652021,3232.067831,3134.798198,3198.484468,1.183567e+10,3.706587e+06,3.834176e+11,0
2,2024-04-20 00:00:00+00:00,3146.652021,3232.067831,3134.798198,3198.484468,1.183567e+10,3.706587e+06,3.834176e+11,0
3,2024-04-19 00:00:00+00:00,3146.652021,3232.067831,3134.798198,3198.484468,1.183567e+10,3.706587e+06,3.834176e+11,0


In [69]:
bigger_preds['date'] = pd.to_datetime(bigger_preds['date'])

upload_data(bigger_preds, 'test_eth_ohlc_predictions', 1)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 100.00% |██████████| Rows 4/4 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: test_eth_ohlc_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/582805/jobs/named/test_eth_ohlc_predictions_1_offline_fg_materialization/executions


In [64]:
eth_yesterday['date'] = pd.to_datetime(eth_yesterday['date'])

upload_data(eth_yesterday, 'test_eth_ohlc_predictions', 1)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

KeyboardInterrupt: 

In [71]:
previous_predictions = pull_data('test_eth_ohlc_predictions', 1, 'test_eth_ohlc_predictions', 1)

previous_predictions


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (0.64s) 


,date,open,high,low,close,volume,volume_eth,market_cap,pred_tmw_1_0_percent_increase_binary
2,2024-04-19 00:00:00+00:00,3146.652021,3232.067831,3134.798198,3198.484468,1.183567e+10,3.706587e+06,3.834176e+11,0
3,2024-04-20 00:00:00+00:00,3146.652021,3232.067831,3134.798198,3198.484468,1.183567e+10,3.706587e+06,3.834176e+11,0
1,2024-04-21 00:00:00+00:00,3146.652021,3232.067831,3134.798198,3198.484468,1.183567e+10,3.706587e+06,3.834176e+11,0
0,2024-04-22 00:00:00+00:00,3146.652021,3232.067831,3134.798198,3198.484468,1.183567e+10,3.706587e+06,3.834176e+11,0


%3|1713826068.995|FAIL|rdkafka#consumer-12| [thrd:ssl://18.191.189.169:9092/bootstrap]: ssl://18.191.189.169:9092/2: Receive failed: SSL transport error: Operation timed out (after 282220ms in state UP)
%5|1713826085.403|REQTMOUT|rdkafka#producer-11| [thrd:ssl://52.14.0.221:9092/bootstrap]: ssl://52.14.0.221:9092/1: Timed out MetadataRequest in flight (after 60053ms, timeout #0)
%4|1713826085.403|REQTMOUT|rdkafka#producer-11| [thrd:ssl://52.14.0.221:9092/bootstrap]: ssl://52.14.0.221:9092/1: Timed out 1 in-flight, 0 retry-queued, 0 out-queue, 0 partially-sent requests
%3|1713826085.404|FAIL|rdkafka#producer-11| [thrd:ssl://52.14.0.221:9092/bootstrap]: ssl://52.14.0.221:9092/1: 1 request(s) timed out: disconnect (after 340860ms in state UP)
%3|1713826086.936|FAIL|rdkafka#consumer-12| [thrd:ssl://3.147.195.6:9092/bootstrap]: ssl://3.147.195.6:9092/3: Receive failed: SSL transport error: Operation timed out (after 661333ms in state UP)
